### 1. What are the unique property or room types in the dataset?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_excel('Airbnb_Open_Data_Set.xlsx', sheet_name='in')
df.columns = [c.strip().lower() for c in df.columns]
room_cols = ['room_type','room type','property_type','property type']
room_col = next((c for c in room_cols if c in df.columns), None)
if room_col is None:
    print('No room/property type column found')
else:
    uniq = df[room_col].dropna().astype(str).str.strip().unique()
    print(pd.Series(uniq))
    plt.figure(figsize=(6.5,4))
    df[room_col].astype(str).str.strip().value_counts().head(15).plot(kind='bar')
    plt.title('Top room/property types')
    plt.xlabel('Type')
    plt.ylabel('Count')
    plt.tight_layout()
    plt.show()
print('Listed unique types and plotted top counts')

### 2. Which neighborhood has the highest number of listings?

In [ ]:
neigh_cols = ['neighbourhood','neighborhood','neighbourhood_cleansed','neighborhood_cleansed']
neigh_col = next((c for c in neigh_cols if c in df.columns), None)
if neigh_col is None:
    print('No neighborhood column found')
else:
    counts = df[neigh_col].astype(str).str.strip().value_counts()
    print(counts.head(20))
    plt.figure(figsize=(7,4))
    counts.head(15).plot(kind='bar', color='#4C78A8')
    plt.title('Listings by neighborhood (top 15)')
    plt.xlabel('Neighborhood')
    plt.ylabel('Listings')
    plt.tight_layout()
    plt.show()
print('Computed neighborhood listing counts and chart')

### 3. Which boroughs or areas have the highest average prices?

In [ ]:
price_cols = ['price','daily_price','avg_price']
price_col = next((c for c in price_cols if c in df.columns), None)
if price_col is None:
    print('No price column found')
else:
    df[price_col] = pd.to_numeric(df[price_col], errors='coerce')
area_cols = ['borough','city','state','neighbourhood_group','neighborhood_group']
area_col = next((c for c in area_cols if c in df.columns), None)
if price_col is None or area_col is None:
    print('Insufficient columns for area price analysis')
else:
    avg_price = df.groupby(df[area_col].astype(str).str.strip())[price_col].mean().sort_values(ascending=False)
    print(avg_price.head(20))
    plt.figure(figsize=(7.5,4.5))
    avg_price.head(10).plot(kind='bar', color='#72B7B2')
    plt.title('Avg price by area (top 10)')
    plt.xlabel('Area')
    plt.ylabel('Average price')
    plt.tight_layout()
    plt.show()
print('Calculated and plotted avg prices by area')

### 4. Is there a relationship between construction year and listing price?

In [ ]:
year_cols = ['year_built','construction_year','built_year']
year_col = next((c for c in year_cols if c in df.columns), None)
if price_col is None or year_col is None:
    print('Insufficient columns for year vs price')
else:
    df[year_col] = pd.to_numeric(df[year_col], errors='coerce')
    dfx = df[[year_col, price_col]].dropna()
    dfx = dfx[(dfx[year_col] >= 1800) & (dfx[year_col] <= 2030)]
    dfx = dfx[dfx[price_col] >= 0]
    if len(dfx) > 0:
        dfx = dfx.sort_values(year_col)
        plt.figure(figsize=(6.8,4.5))
        sns.scatterplot(data=dfx.sample(min(len(dfx), 5000), random_state=42), x=year_col, y=price_col, alpha=0.3, s=15)
        sns.regplot(data=dfx, x=year_col, y=price_col, scatter=False, color='#E45756')
        plt.title('Price vs Construction year')
        plt.tight_layout()
        plt.show()
        dfx['year_bin'] = pd.cut(dfx[year_col], bins=list(range(1800, 2031, 10)))
        bin_avg = dfx.groupby('year_bin')[price_col].mean()
        print(bin_avg.dropna().head(25))
        plt.figure(figsize=(6.8,4))
        plt.plot(range(len(bin_avg.dropna())), bin_avg.dropna().values, marker='o')
        plt.xlabel('Construction year bins')
        plt.ylabel('Avg price')
        plt.title('Avg price by construction year bin')
        plt.tight_layout()
        plt.show()
    else:
        print('No usable rows for year vs price')
print('Analyzed construction year vs price')

### 5. Who are the top hosts by number of listings?

In [ ]:
host_cols = ['host_name','host name']
host_col = next((c for c in host_cols if c in df.columns), None)
host_count_cols = ['calculated_host_listings_count','host_listings_count','host listings count']
host_count_col = next((c for c in host_count_cols if c in df.columns), None)
if host_count_col is None:
    print('No host listings count column found')
else:
    df[host_count_col] = pd.to_numeric(df[host_count_col], errors='coerce')
    if host_col is not None:
        top_hosts = df.groupby(df[host_col].astype(str).str.strip())[host_count_col].max().sort_values(ascending=False).head(10)
        print(top_hosts)
        plt.figure(figsize=(7,4))
        top_hosts.plot(kind='bar', color='#F58518')
        plt.title('Top 10 hosts by calculated listings count')
        plt.xlabel('Host')
        plt.ylabel('Listings count')
        plt.tight_layout()
        plt.show()
    else:
        top_hosts = df[host_count_col].sort_values(ascending=False).head(10)
        print(top_hosts)
print('Listed top hosts')

### 6. Are hosts with verified identities more likely to receive positive reviews?

In [ ]:
from scipy import stats
verified_col = 'host_identity_verified' if 'host_identity_verified' in df.columns else ('host identity verified' if 'host identity verified' in df.columns else None)
rating_cols = ['review_scores_rating','review scores rating','review_score','review scores value','review_scores_value']
rating_col = next((c for c in rating_cols if c in df.columns), None)
if rating_col is None:
    print('No review rating column found')
else:
    df[rating_col] = pd.to_numeric(df[rating_col], errors='coerce')
if verified_col is not None and rating_col is not None:
    vals = df[rating_col]
    if vals.dropna().between(0, 5).mean() > 0.9:
        df['rating_100'] = vals * 20.0
    else:
        df['rating_100'] = vals
    grp = df[[verified_col, 'rating_100']].dropna()
    if grp[verified_col].dtype == object:
        grp[verified_col] = grp[verified_col].astype(str).str.lower().str.strip()
    a = grp.loc[grp[verified_col].isin(['t','true','yes','y','1','verified']), 'rating_100']
    b = grp.loc[~grp[verified_col].isin(['t','true','yes','y','1','verified']), 'rating_100']
    if len(a) > 1 and len(b) > 1:
        tstat, pval = stats.ttest_ind(a, b, equal_var=False, nan_policy='omit')
        print(a.mean())
        print(b.mean())
        print(pval)
        plt.figure(figsize=(6,4))
        sns.kdeplot(a, label='Verified', fill=True, alpha=0.3)
        sns.kdeplot(b, label='Not verified', fill=True, alpha=0.3)
        plt.legend()
        plt.title('Ratings distribution by host verification')
        plt.tight_layout()
        plt.show()
    else:
        print('Insufficient data for t-test')
print('Checked verification vs reviews')

### 7. Is there a correlation between the price of a listing and its service fee?

In [ ]:
air = pd.read_excel('Airbnb_Open_Data_Set.xlsx', sheet_name='in')
air.columns = [c.strip().lower() for c in air.columns]
pcol = 'price' if 'price' in air.columns else None
fcol_candidates = ['service fee','service_fee','service fees','service_fees']
fcol = next((c for c in fcol_candidates if c in air.columns), None)
if pcol is None or fcol is None:
    print('Missing price or service fee column')
else:
    air[pcol] = pd.to_numeric(air[pcol], errors='coerce')
    air[fcol] = pd.to_numeric(air[fcol], errors='coerce')
    dfc = air[[pcol, fcol]].dropna()
    p_cap = dfc[pcol].quantile(0.99)
    f_cap = dfc[fcol].quantile(0.99)
    dfc = dfc[(dfc[pcol] >= 0) & (dfc[pcol] <= p_cap) & (dfc[fcol] >= 0) & (dfc[fcol] <= f_cap)]
    if len(dfc) > 1:
        pear = dfc[pcol].corr(dfc[fcol], method='pearson')
        spear = dfc[pcol].corr(dfc[fcol], method='spearman')
        print(pear)
        print(spear)
        plt.figure(figsize=(6.5,4.5))
        sns.regplot(data=dfc, x=pcol, y=fcol, scatter_kws={'alpha':0.25, 's':12}, line_kws={'color':'#E45756'})
        plt.xlabel('Price')
        plt.ylabel('Service fee')
        plt.title('Service fee vs Price')
        plt.tight_layout()
        plt.show()
        dfc['price_bin'] = pd.qcut(dfc[pcol], 10, duplicates='drop')
        bin_avg = dfc.groupby('price_bin')[fcol].mean()
        print(bin_avg)
        plt.figure(figsize=(6.5,4))
        sns.lineplot(x=range(len(bin_avg)), y=bin_avg.values, marker='o')
        plt.xlabel('Price decile (low to high)')
        plt.ylabel('Avg service fee')
        plt.title('Avg service fee by price decile')
        plt.tight_layout()
        plt.show()
    else:
        print('Insufficient rows for correlation')
print('Computed correlations and displayed two charts')